# 第04章 — パーティクルシステム

- パーティクルは位置・速度・寿命を持ち、エミッタが多数生成する。
- 重力や風などの力を一様にかけると煙や雨のようなまとまりが出る。
- 寿命が尽きた粒子を再利用すると計算が効率的。
- ばらつきには乱数を使うが、分布を制御すると説得力が増す。

試してみよう: 重力や初速のばらつきを調整して、噴水から煙のような挙動まで変化させる。

In [1]:
# パーティクル演算に使うベクトルクラス
import random

class Vector:
    def __init__(self, x=0.0, y=0.0):
        self.x, self.y = float(x), float(y)

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __mul__(self, k):
        return Vector(self.x * k, self.y * k)

    __rmul__ = __mul__

    def __repr__(self):
        return f"Vector({self.x:.2f}, {self.y:.2f})"


In [2]:
# 寿命付きパーティクルの定義
class Particle:
    def __init__(self, origin):
        self.pos = Vector(*origin)
        self.vel = Vector(random.uniform(-1, 1), random.uniform(0, 2))
        self.lifespan = 80

    def update(self, gravity=Vector(0, -0.05)):
        self.vel += gravity
        self.pos += self.vel
        self.lifespan -= 1

    def alive(self):
        return self.lifespan > 0


In [3]:
# 重力と追加スポーンを含む簡単なパーティクルシミュレーション
particles = [Particle((0, 0)) for _ in range(5)]

for frame in range(8):
    for p in particles:
        p.update()
    particles = [p for p in particles if p.alive()]
    if frame % 2 == 0:
        particles.append(Particle((0, 0)))
    print(f"frame {frame}: count={len(particles)} positions={[p.pos for p in particles[:3]]}")


frame 0: count=6 positions=[Vector(0.37, 1.89), Vector(0.45, 1.01), Vector(0.53, 1.83)]
frame 1: count=6 positions=[Vector(0.75, 3.73), Vector(0.90, 1.96), Vector(1.05, 3.61)]
frame 2: count=7 positions=[Vector(1.12, 5.51), Vector(1.36, 2.87), Vector(1.58, 5.34)]
frame 3: count=7 positions=[Vector(1.49, 7.25), Vector(1.81, 3.72), Vector(2.11, 7.01)]
frame 4: count=8 positions=[Vector(1.86, 8.94), Vector(2.26, 4.53), Vector(2.64, 8.64)]
frame 5: count=8 positions=[Vector(2.24, 10.58), Vector(2.71, 5.28), Vector(3.16, 10.22)]
frame 6: count=9 positions=[Vector(2.61, 12.17), Vector(3.16, 5.99), Vector(3.69, 11.75)]
frame 7: count=9 positions=[Vector(2.98, 13.70), Vector(3.61, 6.64), Vector(4.22, 13.23)]


### 追加例: 風のガストを受けるパーティクル群

In [4]:
# 突風を受けるパーティクル群を簡易的にシミュレーション
import random

class Particle:
    def __init__(self):
        self.x, self.y = 0.0, 0.0
        self.vx = random.uniform(-0.5, 0.5)
        self.vy = random.uniform(1.0, 2.0)
        self.life = 40

    def step(self, wind=0.0):
        self.vx += wind
        self.vy += -0.05
        self.x += self.vx
        self.y += self.vy
        self.life -= 1

particles = [Particle() for _ in range(8)]
for frame in range(6):
    gust = 0.2 if frame in (2, 3) else 0.0
    for p in particles:
        p.step(wind=gust)
    particles = [p for p in particles if p.life > 0]
    print(f"frame {frame}: positions={[ (round(p.x,2), round(p.y,2)) for p in particles[:3] ]}")


frame 0: positions=[(-0.2, 1.69), (0.4, 1.92), (0.0, 1.92)]
frame 1: positions=[(-0.41, 3.34), (0.79, 3.8), (0.0, 3.78)]
frame 2: positions=[(-0.41, 4.93), (1.39, 5.62), (0.2, 5.6)]
frame 3: positions=[(-0.21, 6.47), (2.18, 7.39), (0.6, 7.37)]
frame 4: positions=[(-0.02, 7.97), (2.98, 9.12), (1.0, 9.09)]
frame 5: positions=[(0.18, 9.41), (3.77, 10.79), (1.4, 10.75)]
